<a href="https://colab.research.google.com/github/anushreehede/EmailReceipts/blob/master/522_HW4_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 4: Natural Language Processing

**Overview**: In this homework you will be working with the Amazon Reviews Dataset. This dataset contains hundreds of thousands of reviews, each with it's raw text, it's summary and a score from 1-5 on the rating corresponding to the review. In this assignment you will be creating a multitude of discriminative models to classify the rating from the text of the review, from simple non-deep learning baseline approaches to creating multiple different deep recurrent neural networks, using attention and transfer learning to boost classfication accuracy. Past that, you will create a generative model to generate summaries of a given review from scratch!


**Collaboration Policy:** This homework assignment is meant to be done in **groups of 2**. You may work on this individually, but be warned that this homework assignment is long and will be difficult to do alone. We highly recommend you form groups. Since this is a group assignment, please read over the collaboration policy again, posted [here](https://www.seas.upenn.edu/~cis522/syllabus.html).



# Q0. Setting up (0 pts)

## Connect to Google Drive

Run the code snippet below to connect to Google Drive.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

## Utility Functions

The function below is a utility function for plotting Confusion Matrices. We highly recommend that you use it for your analysis, but if you have an easier way of plotting confusion matrices / would like to do it yourself, feel free to do so instead.

In [0]:
from textwrap import wrap
import re
import itertools
import matplotlib
import numpy as np
from sklearn.metrics import confusion_matrix


# Credits - https://stackoverflow.com/questions/41617463/tensorflow-confusion-matrix-in-tensorboard
def plot_confusion_matrix(correct_labels, predict_labels, labels, display_labels, title='Confusion matrix', tensor_name = 'MyFigure/image', normalize=False):
  ''' 
  Parameters:
      correct_labels                  : These are your true classification categories.
      predict_labels                  : These are you predicted classification categories
      labels                          : This is a lit of labels which will be used to display the axix labels
      title='Confusion matrix'        : Title for your matrix
      tensor_name = 'MyFigure/image'  : Name for the output summay tensor

  Returns:
      summary: TensorFlow summary 

  Other itema to note:
      - Depending on the number of category and the data , you may have to modify the figzie, font sizes etc. 
      - Currently, some of the ticks dont line up due to rotations.
  '''
  cm = confusion_matrix(correct_labels, predict_labels, labels=labels)
  if normalize:
      cm = cm.astype('float')*10 / cm.sum(axis=1)[:, np.newaxis]
      cm = np.nan_to_num(cm, copy=True)
      cm = cm.astype('int')

  np.set_printoptions(precision=2)
  ###fig, ax = matplotlib.figure.Figure()

  fig = matplotlib.pyplot.figure(figsize=(2, 2), dpi=320, facecolor='w', edgecolor='k')
  ax = fig.add_subplot(1, 1, 1)
  im = ax.imshow(cm, cmap='Oranges')

  classes = [re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', x) for x in display_labels]
  classes = ['\n'.join(wrap(l, 40)) for l in classes]

  tick_marks = np.arange(len(classes))

  ax.set_xlabel('Predicted', fontsize=7)
  ax.set_xticks(tick_marks)
  c = ax.set_xticklabels(classes, fontsize=4, rotation=-90,  ha='center')
  ax.xaxis.set_label_position('bottom')
  ax.xaxis.tick_bottom()

  ax.set_ylabel('True Label', fontsize=7)
  ax.set_yticks(tick_marks)
  ax.set_yticklabels(classes, fontsize=4, va ='center')
  ax.yaxis.set_label_position('left')
  ax.yaxis.tick_left()

  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      ax.text(j, i, format(cm[i, j], 'd') if cm[i,j]!=0 else '.', horizontalalignment="center", fontsize=6, verticalalignment='center', color= "black")
  fig.set_tight_layout(True)
  matplotlib.pyplot.show()

  return

### Test Infrastructure

Run the two code snippets below to instantiate our infrastructure for this assignment.

In [0]:
import os
import dill
import pandas as pd

test_csv_path = "/content/Homework4_Resources/test_release.csv"

if not os.path.isdir(repo_path):
  !git clone https://github.com/CIS522/Homework4_Resources.git

test_csv = pd.read_csv(test_csv_path)

def generate_test_predictions(generate_predictions_func):
  return generate_predictions_func(test_csv)

In [0]:
test_case_1b        = None
test_case_1c        = None
test_case_3e_rnn    = None
test_case_3e_lstm   = None
test_case_3e_gru    = None
test_case_3e_bilstm = None
test_case_4e        = None
test_case_5b        = None

# Data Loading and Pre-processing

In this assignment, you'll be working with the Amazon Reviews Dataset. We've provided you with the train / validate splits. In the code cell below, load in the splits using Pandas.

## Load Splits

In [0]:
# TODO: INSERT CODE HERE

# Q1. Baseline (15 pts)

For any deep learning task, it's good to have a baseline approach to compare against so you can determine if your deep learning approach is truly doing anything extraordinary, or in some cases, is even worse than simpler approaches. In this question we will have you implement a very simple baseline approach for classifying Amazon Reviews.

**Baselines** You will be implementing two simple baseline approaches:

*Approach 1:* For each sentence, process each word individually and note whether it is positive or negative. You will calculate the ratio of positive words to positive + negative words in the sentence, and threshold it to get a prediction between 1-5 of the review's star rating. 

*Approach 2:* In this approach you will feed in the number of positive and negative words in each sentence into a logistic regression classifier to get a classification between 1-5.



## Q1a Processing the Data

Before implementing the baselines, you must first process the data! Look through the Amazon Reviews dataset and consider the headers. For this question, all you will need are the "Text" and "Score" fields. 

Before we process the data, we must note two things:

**How do we get the connotation?** To determine if a word has a "positive" or a "negative" connotation we will be using the [MPQA subjectivity lexicon](https://mpqa.cs.pitt.edu/lexicons/subj_lexicon/) (which we've provided to you as a .tff file in this assignment). If you open the .tff file in a text editor, you'll see that each line has the word, preceded by "word1=" and a polarity of the form "priorpolarity=negative" for a negatively connoted word for instance. You should parse the tff file to create a dictionary that goes from word to it's polarity (positive or negative)

**How do we handle different forms of words?** In this tff file, we note that there's a lot of words, but how do we account for different forms of words? For instance, how do we account for the plural form of a noun, or a different conjugation of a verb? In NLP it is very common to account for this by "lemmatizing" a word, which essentially means to convert words to a "base" form so they can be analyzed more easily. In this classification task, this is useful since we don't care about how a word is conjugated, just what the connotation of a word is.

**Q1a (1 pts):** In this question, create a dictionary from the .tff file such that the keys are the lemmatized forms of the word, and the value is the polarity of the word (if the word is positive, let its value be 1, if it's negative let it be 0). We've initialized NLTK and specifically the lemmatizer for you below. Do ```lemmatizer.lemmatize(word)``` to lemmatize a specific word. In your writeup, give three examples of words which are different than their lemmatized forms (i.e. are not the exact same text).



In [0]:
import nltk
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
nltk.download('punkt') # Download this as this allows you to tokenize words in a string.
lemmatizer = WordNetLemmatizer()

**Thresholding Baseline**

Now that you've created this dictionary, go through your training dataset and for each review, compute the following ratio: $$ratio = \frac{positive}{positive + negative}$$ Where $positive$ is the number of words with positive polarity in the review, and $negative$ is the number of words with negative polarity in the dataset. 

**Notes:** You can tokenize the text into individual words by using ```nltk.word_tokenize(text)```. Don't forget to lemmatize each word as well! You can treat any word not in the MPQA subjectivity lexicon as having a neutral sentiment. For the case where $positive + negative = 0$ feel free to handle it however you like (i.e. always have it predict a certain class, etc.). Describe how you handled this in your writeup for **Q1d**.

**Q1b (2 pts; 2 pts autograder):** Using the validation set, create a series of thresholds such that you can threshold this ratio to produce your review predictions between 1 and 5. Calculate the F1 score of your predictions versus the ground truth scores (setting average to **macro**). You may use SkLearn to calculate the F1 Score. In your writeup, describe how you chose these thresholds and what the resulting macro average F1 Score is for your training set and validation set.

In [0]:
# TODO: INSERT CODE HERE

**Q1c (3 pts; 3pts autograder):** Now, train a logistic regression classifier, where the inputs to the logistic regression classifier are the number of positively and negatively connoted words in the sentence. Tune the hyperparameters using the validation set. Create a test set from the training set, train the logistic regression using the hyperparameters obtained from tuning on the validation set, and then report your test, training, and validation set macro-averaged F1 Scores in your writeup. Discuss how you tuned hyperparameters in your writeup.

In [0]:
# TODO: INSERT CODE HERE

**Q1d (4 pts):** Compare the results from the logistic regression baseline and the thresholding baseline. Plot a confusion matrix for both and compare the results / put the confusion matrices in the writeup (feel free to use the utility function we provided above).

**Submitting your solutions:** As in the previous homeworks, implement the below header that takes in a Pandas dataframe and returns the predictions of review scores for each of the Amazon reviews.

In [0]:
# TODO: INSERT CODE HERE

In [0]:
test_case_1b = generate_test_predictions(generate_baseline_predictions)            # 2 pts
test_case_1c = generate_test_predictions(generate_logistic_regression_predictions) # 3 pts

# Q2. Featurizing the dataset using torchtext (10 pts)

As you saw, data for computer vision is relatively easy to format, simply resize or pad it to a standard format and you can feed it into your neural network. However, text data typically requires more processing before it can be used seamlessly with PyTorch. Thankfully, Pytorch created torchtext, which (similar to torchvision) provides the preprocessing tools to work with textual data.

In this question, we will create and featurize a torchtext dataset with Fields, a data structure that can automatically featurize text with its corresponding word embeddings. We will be using the same dataset for the review classification and the summary generation task. Now, lets load the data required only for the review classification task.





## Q2a. Create torchtext data fields (0 pts)

First, we create TorchText "Fields". Fields describe how to process the input data so it can later be converted to a tensor. For the classification task, we need only two fields from the data - the review text and the review score. Create two torchtext data fields using data.Field from torchtext.data. The documentation for torchtext is [here](https://torchtext.readthedocs.io/en/latest/).


*   A sequential field named TEXT for the review text.  (*Hint: since this is natural language, this is sequential data.  Also, use the "spacy" tokenizer to tokenize the text. This tokenizer handles converting a string into a series of single words. If you're curious how the spaCy tokenizer works, check out its website [here](https://spacy.io/usage/linguistic-features#tokenization). Also to standardize the tokens, we highly recommend making it lowercase.*)
*   A non-sequential field named SCORE for the scores.  (*Hint: since this is a categorical variable, it is not sequential.  Furthermore, it does not require a vocabulary since there are no words to embed.*)

In [0]:
# TODO: INSERT CODE HERE

## Q2b. Create a Tabular Dataset (3 pts)

Now that we've defined the two Fields, which determine how to process our data, let's actually produce a dataset with our data! TorchText provides a class called a ```TabularDataset``` which reads in the input data (in this case in the form of a CSV file) and produces a Dataset from it.

Now, look at the [documentation](https://torchtext.readthedocs.io/en/latest/data.html#tabulardataset) for `data.TabularDataset` and create two tabular datasets, one for your training data and one for your testing data.  You will need the paths to the training and validation datasets, the format, and the mapping from columns to fields.  
The mapping from columns to fields should be provided as a list of tuples where the first tuple element should be the name of the column in the csv and the second element should be the corresponding Field created in the previous step. Columns which you do not use/require will need to be indicated by using *None* for that column in place of the field.
Note that the first column in the csv's should not be a field. Also note that while loading the dataset, we would want to ignore the first row (which represents the header).


**NOTE :** This step creates 2 torchtext objects for each row in the dataset, so it will take some time (between 2-10 minutes).

In [0]:
# TODO: INSERT CODE HERE

**Q2b (3 pt)** Now, use the dataset created above to print 5 examples of reviews and their rating. Each example should be a tokenized list of the review. Include a screenshot of the output to the write-up


In [0]:
# TODO: INSERT CODE HERE

## Q2c. Build Vocab (3 pts)

Now that we have our text parsed, how do we actually represent our data as tensors? To do this, we use word embeddings, which we highly recommend reading about if you do not understand what they are. Using these word embeddings, we then build our vocabulary with GloVE, a word embedding similar to Word2Vec.  Create a vocabulary from the Field created in Q2a using the train dataset (*Hint: look at the documentation for Field.build_vocab*).  Use the glove.6B.300d word embedding, and save the vocbulary.  The first time you run this, it will take roughly 5-10 minutes to download.  The pretrained model will then be stored in the ./.vector_cache folder, and rerunning this command will take negligible time.

Note that you can pass more than one datafield to the `build_vocab` function. Build a single vocab for both, the training data and validation data.

**Q2c (3 pts)** Print four properties of the `vocab` - ('freqs', 'itos', 'stoi', 'vectors'). Include a screenshot of the printouts in your writeup. Report the size of the vocabulary.

In [0]:
# TODO: INSERT CODE HERE

## Q2d. Create an iterator for the dataset (0 pts)

Now, create a `BucketIterator` for the training data and the validation data. The `BucketIterator` is equivalent to the `DataLoader`. It splits the data into batches which can be used with recurrent models. This will create batches and add `<pad>` token so that each batch will have the same length. `BucketIterator` creates batches such that the amount of padding is minimum. You will need to sort each batch by the length of the text in decreasing order (this is required for passing the batch through recurrent layers). Look at the documentation of `torchtext.data.BucketIterator` and check the `sort_key` parameter.

In [0]:
# TODO: INSERT CODE HERE

## Q2e. Get frequency distribution per class (4 pts)

**Q2e (4 pts):** Now, print the frequency distribution for each class for the training data and the validation data. Report the frequencies in your write-up.

In [0]:
# TODO: INSERT CODE HERE

# Q3. Recurrent Amazon Reviews Classifier (25 pts)

We will now build the model for predicting the rating given the review. 

As we are working with text, we will first build a model which uses recurrent layers. Define a model for the classifier which uses *RNN*, *GRU*, *LSTM* and *BiLSTM* layers to predict the rating. You can make four different models or can make a single model with a `mode` parameter which defines which of the four recurrent layers to use. We highly recommend the latter approach, it will make the process much quicker.

**Define the basic recurrent model**

The model should have a `nn.Embedding` layer whose weights should be initialized to the pretrained embeddings downloaded above. The hidden state output from the last timestep should be passed to one (or more) fully connected layers to produce the output. For the *BiLSTM* model, you can choose to either add or concatenate the hidden state for the two directions (feel free to experiment with both and pick the one which works the best).

For this assignment, we will use the *F1 score* to monitor performance of the model. Check `sklearn.metrics.f1_score` to calculate the F1 score.

**NOTE:** Also we recommend you use `nn.utils.rnn.pad_packed_sequence` before passing the input to the recurrent layers. This will make sure the padding is ignored for every sample in a batch and the output will be from the actual last token for every sequence.

In [0]:
class ReviewClassifier(nn.Module):
    """ 
    Review Classifier
  
    Here we give you the method header for the Review Classifier to make the process easier.
    You don't have to use this exact format, feel free to change the parameters as much as you
    would like, these are simply the parameters that our solution had.
  
    Parameters: 
    mode (string): Type of recurrent layer being used. Types are ['rnn', 'lstm', 'gru', 'bilstm']
    output_size (int): Size of the last layer for classification (hint: how many classes do you have?)
    hidden_size (int): Length of your hidden state vector
    vocab_size (int): Length of your vocab (can get this by doing len(TEXT.vocab))
    embedding_length (int): Dimension of your word embedding vector (hint: look at part 2c)
    word_embeddings (Tensor): All of the word embeddings generated. Can get this from TEXT.vocab.vectors
    """
  def __init__(self, mode, output_size, hidden_size, vocab_size, embedding_length, word_embeddings):
    super(ReviewClassifier, self).__init__()

    if mode not in ['rnn', 'lstm', 'gru', 'bilstm']:
      raise ValueError("Choose a mode from - rnn / lstm / gru / bilstm")

  def forward(self, text, text_lengths):
    # TODO
    return None

**Train the Recurrent Models**

**Q3b (2 pts):** Train the model defined above with *LSTM* as the recurrent layer. Use the standard `nn.CrossEntropyLoss` as the loss function.  Report the final training F1 score and the final training loss of the model (trained only on the training set). Use the trained model to evaluate on the validation data. Report the validation F1 score. Use the `plot_confusion_matrix` function to display the confusion matrix for the validation set. Report this confusion matrix to your write-up. Report your final hyper-parameter choices in your writeup as well as your hyper-parameter tuning process.

In [0]:
# TODO: INSERT CODE HERE

**Q3c (2 pts):** What issue do you find after looking at the confusion matrix? What according to you is the reason for this difference? What would you do to fix it? *Hint:* Note what you observed in Q2e. The fix has something to do with the loss function.

In [0]:
# TODO: INSERT CODE HERE

**Q3d (2 pts):** Now redo 3b except with the fix for the problem that you noted in Q3c. Report all the same things in your writeup, and explain how adding this fix impacted results. Use the confusion matrix and the F1 Score in your analysis.

In [0]:
# TODO: INSERT CODE HERE

**Q3e (10 pts):** Train the model with the other three recurrent layers. Plot the training loss and training F1 score for all four (including the one trained in Q3b-d) models on the same graph. (You will provide two plots, one with the training loss for all 4 models and the other with the training F1 score for all 4 models). Also report the confusion matrix for all the four trained models for the validation set. What do you observe? Compare the four models and provide a rough intuition as to why the models performed the way they did. 

In [0]:
# TODO: INSERT CODE HERE

**Submitting your solutions:** As in the previous homeworks, implement the below header that takes in a Pandas dataframe and returns the predictions of review scores for each of the Amazon reviews.

In [0]:
# Inputs: The input to this function is a pandas dataframe
# Outputs: Return a **numpy** array with your score predictions. 
# **Note** that these score predictions should be between *1-5*.

def generate_rnn_predictions(input_df):
  # TODO: Write your code here
  return None

def generate_gru_predictions(input_df):
  # TODO: Write your code here
  return None

def generate_lstm_predictions(input_df):
  # TODO: Write your code here
  return None

def generate_bilstm_predictions(input_df):
  # TODO: Write your code here
  return None

In [0]:
test_case_3e_rnn    = generate_test_predictions(generate_rnn_predictions)        # 1 pts  
test_case_3e_gru    = generate_test_predictions(generate_gru_predictions)        # 2 pts
test_case_3e_lstm   = generate_test_predictions(generate_lstm_predictions)       # 3 pts
test_case_3e_bilstm = generate_test_predictions(generate_bilstm_predictions)     # 3 pts

# Q4. Classifier with self-attention (30 pts)

Now, we will implement the Review Classifier with self-attention. Self-attention is a technique where we score the input sequence hidden states with respect to the final hidden representation we use for classification. 

Attention is a very useful technique used in all of deep learning to figure out what parts of the input are more important for producing the output. This helps us use a more relevant feature representation of the input with respect to the end goal. In NLP applications, we typically want to know which words (or tokens) in the input sequence are important for our task. In this assignment, we are interested to figure out what words in the review text are the most important to predict the rating. We call it self-attention because the words from the same input sentence are attended on for the classification.

You can read about attention in more detail from [here](https://medium.com/datadriveninvestor/attention-in-rnns-321fbcd64f05).

Attention was originally introduced for encoder-decoder architectures used for machine translation in [this paper](https://arxiv.org/pdf/1409.0473.pdf).



## Q4a. Define the attention module

Define an `Attention` module (just like you define any model). For this assignment, we will be implementing *additive attention*. So, the inputs to the attention will be passed through fully connected layers and finally through a softmax to output the `attention_scores`. The following figure summarizes the procedure to compute the `attention_scores`. Implement the `forward()` of the `Attention` module as per the figure.

![attention](https://i.imgur.com/uFdThV6.png)

To implement self-attention we will first define an `Attention` module. This module will take as input the final representation with respect to which we want to attend on the input sequence (for this assignment, the last hidden state) and the output of the recurrent layer from each timestep and outputs `attention_scores` which is a probability distribution over each token in the input sequence representing the importance of each token for the final prediction.

In [0]:
class Attention(nn.Module):
  def __init__(self, hidden_size):
    super(Attention, self).__init__()

  def forward(self, hidden, encoder_outputs):
    """ 
    Parameters: 
    hidden (vector): Final hidden state from the input sequence
    encoder_outputs (tensor): Hidden state produced from each of the input sequence tokens
    """
    return None


## Q4b. Define the classifier model with attention

Now, define a model which uses the `Attention` module defines above. This model will be the same as the one defined in section Q3 except that you will compute a `context` vector (calculated as a weighted average of the outputs for each timestep with weights from the `Attention` module) and then concatenate this vector with the last hidden state vector . The context vector concatenated with the last hidden state should be passed through one (or more) fully connected layers to generate the final predictions.

**NOTE:** Pay careful attention to the dimensions of the output. The recurrent layers expect input with the batch size at the second dimension and sequence length as the first dimension. The output also follows the same rule.

**NOTE:** Also we recommend you use `nn.utils.rnn.pad_packed_sequence` before passing the input to the recurrent layers. This will make sure the padding is ignored for every sample in a batch and the output will be from the last batch 

In [0]:
class ReviewClassifierWithAttention(nn.Module):
  def __init__(self, mode, output_size, hidden_size, vocab_size, embedding_length, word_embeddings):
    super(ReviewClassifierWithAttention, self).__init__()

    if mode not in ['rnn', 'lstm', 'gru', 'bilstm']:
      raise ValueError("Choose a mode from - rnn / lstm / gru / bilstm")

  def forward(self, text, text_lengths):
    # TODO
    return None

## Q4c. Training the attention model

Train the model defined above using the recurrent layer which performed the best in the previous section. (You can choose to tune the hyperparameters - recurrent layers, hidden size, etc. also to pick the best model with self attention). 

**Q4c (5 pts):** Report the final accuracy, F1 score and loss for the training and validation sets. Include the training and validation plots in your write-up. Also plot the confusion matrix for the validation data and include it in your write-up.

**Q4d (5 pts):** How does the model with and without self attention compare? What do you think intuitively explains the difference in performance?

In [0]:
# TODO: INSERT CODE HERE

**Submitting your solutions:** As in the previous homeworks, implement the below header that takes in a Pandas dataframe and returns the predictions of review scores for each of the Amazon reviews.

In [0]:
# Inputs: The input to this function is a pandas dataframe
# Outputs: Return a **numpy** array with your score predictions. 
# **Note** that these score predictions should be between *1-5*.

def generate_attention_predictions(input_df):
  # TODO: Write your code here
  return None

In [0]:
test_case_4e = generate_test_predictions(generate_attention_predictions)         # 20 pts  

# Q5. Transfer Learning using Hugging Face (20 pts)

As you saw in the computer vision homework, transfer learning is a powerful tool to boost the accuracy of a model, especially with limited data. Most transfer learning models for text applications are extremely large and difficult to work with, so to make this easier most use a github repository called [Hugging Face](https://github.com/huggingface) (which we highly recommend checking out!). Getting the data to be in the right format for training can be difficult even with Hugging Face, so for this assignment we'll be using a library built on top of Hugging Face called [Simple Transformers](https://github.com/ThilinaRajapakse/simpletransformers), which makes transfer learning extremely easy.

## Setting up Simple Transformers

First we download the pip package for SimpleTransformers

In [0]:
!pip install simpletransformers

Now, to initialize a classification model, you can do so similarly to how we do it below. Feel free to change the args passed in, however, make sure to keep "fp16" set to false (trust us on this).

In [0]:
from simpletransformers.classification import ClassificationModel
model = ClassificationModel('roberta', 'roberta-base', num_labels=6, args=({'fp16': False, 'reprocess_input_data': True}))

The input format for SimpleTransformers is to have the text data in a columm called "text" and to have the labels as a column called "labels" so we do that below.

Great! Now that you've downloaded the model, let's train with it. To train with it, run the following code snippet below. 

**Note:** This will take a pretty long time so grab a coffee, turn on a Netflix show and watch a few episodes, play a few games of Overwatch, whatever you do to kill time. You can actually run up to two Colab runtimes at once, so you can always duplicate this notebook and keep on working on the homework -- up to you.

You'll note that this will progress in two stages: In the first stage the model converts your input data into features (similar to when you created your TabularDataset earlier). In the second stage, it will actually train with those input features and labels, and display the progress to you. For reference, it took around ~3 hours to run through a single epoch of the dataset (yikes).

**Hint:** We recommend that you make sure this works on a subset of your training data / testing data (like 10% of it) before you run it on the whole dataset so you don't waste lots of time.

In [0]:
# Train the model
model.train_model(train_df_cleaned)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_df_cleaned)

**Q5a (8 pts):** Report your final testing macro average F1 Score as well as confusion matrix in your writeup. Analyze your results in comparison to the recurrent models you  trained previously (including both the self-attention and non self-attention models). Try out at least 2 different models (roberta included) and compare their efficacies in the writeup.


**Submitting your solutions:** As in the previous homeworks, implement the below header that takes in a Pandas dataframe and returns the predictions of review scores for each of the Amazon reviews.

In [0]:
# Inputs: The input to this function is a pandas dataframe
# Outputs: Return a **numpy** array with your score predictions. 
# **Note** that these score predictions should be between *1-5*.

def generate_transfer_learning_predictions(input_df):
  # TODO: Write your code here
  return None

In [0]:
test_case_5b = generate_test_predictions(generate_transfer_learning_predictions) # 12 pts

# **Extra Credit:** Q6. Seq2Seq model (25 pts)

**Note:** The entirety of question 6 is **Extra Credit**. This will be a bit more challenging, especially to get good results, but if you have time we highly recommend trying it out!

In this section, you will implement a Seq2Seq architecture to generate the summary for the Review text. The dataset contains one more attribute which is the summary of the review text (it is actually the title reviewers gave while writing the review and the review text is the description provided).

You will implement the standard *Encoder-Decoder* model where the review text will be given as input to the encoder and the decoder will generate the summary. For simplicity of this assignment, we will implement a basic version of the encoder-decoder architecture and use standard training techniques. More sophisticated versions of the architeture like encoder-decoder attention, teacher-forcing (while training), beam search (for inference), etc. will help the model perform better. But for the purpose of this assignment, we will expect you to implement a basic version of the architecture.



## Q6a. Featuring the dataset using torchtext (for the generative task) (0 pts)

For the text generation task, you will need to redefine the torchtext data fields. Create one torchtext field. Note that, we now need to add start and end tokens to both the text sequences. Look at the `init_token` and `eos_token` parameters when creating the data field.

Then create a tabular dataset for the training and validation set. This time also load the *Summary* field. Note that to ensure we create a single common vocabulary for both - the text and summary, we will use the same data field for both the attributes.

Now build the vocabulary for the the data field you created using both the training and validation datasets created in the previous step. As before you should continue using the glove.6B.300d word embeddings.

Then just as you did in Q1, create a `BucketIterator` for both the training and validation set.

In [0]:
# TODO: INSERT CODE HERE

## Q6a. The Encoder Model (3 pts)

You will start by defining the encoder model. This model will be exactly the same as the one implemented in section Q3. The encoder's input will be the review text and will return an encoded representation of the text (which will be the output of the recurrent layer you use). Use any one of the recurrent layers like the one which performed the best in section Q3 or tune it as a hyperparameter to choose which works the best.

**Q6a (3 pts):** Take a screenshot of your encoder architecture and attach it in your writeup. Explain your rationale for the architecture.

In [0]:
# TODO: INSERT CODE HERE

## Q6b. The Decoder Model (3 pts)

Define the `decoder` model. It will be the same as the `encoder` model with the only difference being there will be an additional fully-connected layer after the output from each token in the sequence. This FC layer will have an output size equal to the vocabulary size.

**Q6b (3 pts):** Take a screenshot of your decoder architecture and attach it in your writeup. Explain your rationale for the architecture.

In [0]:
# TODO: INSERT CODE HERE

## Q6c. The Encoder-Decoder Model (4 pts)

Now, you will define the encoder-decoder model which represents the overall architecture end-to-end. 

The encoder-decoder architeture will behave as follows,


*   Pass the review text as input through the encoder.
*   Initialize the hidden state of the decoder with the last hidden state output from the encoder.
*   The decoder will take the `<start>` token as the input in the first timestep and produce an output one timestep at a time. This means your decoder should be passed only one token at a time and output a single token. The output from the previous timestep will be fed as input in the current timestep.
*   Return the output from the decoder concatenated at each timestep.

While training, the decoder will generate outputs till the maximum summary length for that batch. This means, you will call the decoder in a for loop which runs till the length of the decoder output sequence is equal to the maximum length of the summary (true label) for that batch.

During evaluation, the decoder will produce an output till the decoder outputs the `<end>` token. The batch size has to be 1 during evaluation.

**Q6c (4 pts):** Take a screenshot of your encoder-decoder architecture and attach it in your writeup. Explain your rationale for the architecture.



In [0]:
# TODO: INSERT CODE HERE

## Q6d. Train Seq2Seq (5 pts)

**Q6d (5 pts):** Initialize all the three models with appropriate parameters. Train the Seq2Seq model end-to-end. Use `nn.CrossEntropyLoss` for the decoder output. Report the final training loss. Plot the training loss curve and include it in your write-up.

In [0]:
# TODO: INSERT CODE HERE

## Q6e. Evaluate Seq2Seq (10 pts)

**Q6e (5 pts):** Write a function to run inference on the Seq2Seq model. This function should generate the summary for one review at a time and print it to Tensorboard. Make sure you remove the `<start>`, `<end>` and `<pad>` tokens if any before logging it to tensorboard. In your writeup, report 5 of your best summarizations (i.e. include the raw text, the ground truth summarization, and your summarization). Don't worry, we'll be very generous grading your generated summaries! 

In [0]:
# TODO: INSERT CODE HERE

**Q6f (5 pts):** Analyze your summarizations in the writeup. Are there any words that are repeated? Why do you think the generation is as such? There's no right answers here, respond with whatever you observe from your results.

# Submission Instructions

The deadline for all parts of this assignment is **April 2, 2020 at 11:59pm**.

There are two submissions on Gradescope:
*   HW4 - Natural Language Processing (Writeup)
*   HW4 - Natural Language Processing (Programming)

For *HW4 - Natural Language Processing (Writeup)* submit your written responses to the questions above as the file **"522_HW4_writeup.pdf"**

For *HW4 - Natural Language Processing (Programming)* submit the following files as your submission:
1.   522_HW4_submission.dill
2.   This notebook, named as 522_HW4_notebook.ipynb

**Leaderboard:** When you submit, you can choose to add yourself as part of the leaderboard! Extra credit may be awarded to those that perform particularly well on the leaderboard.

**Gradescope Score:** After you submit, you'll see that all of the classifier questions are out of zero points. To encourage as much hypertuning, we will not be releasing the accuracy cutoffs until after the late deadline has passed.

In [0]:
import dill
import os

file_to_obj_map = {
    "test_case_1b"        : test_case_1b,
    "test_case_1c"        : test_case_1c,
    "test_case_3e_rnn"    : test_case_3e_rnn,
    "test_case_3e_lstm"   : test_case_3e_lstm,
    "test_case_3e_gru"    : test_case_3e_gru,
    "test_case_3e_bilstm" : test_case_3e_bilstm,
    "test_case_4e"        : test_case_4e,
    "test_case_5b"        : test_case_5b
}

tests_path = "522_HW4_submission.dill"
with open(tests_path, 'wb') as pickle_file:
  dill.dump(file_to_obj_map, pickle_file)